Note: the site loaded dynamically, so it was necessary to imitate user actions using the library Selenium

In [140]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import time

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [141]:
df = pd.read_excel("URLs.xlsx", sheet_name="Sheet4")
df

,URL
0,https://ostrovok.ru/hotel/russia/arkhangelsk/m...
1,https://ostrovok.ru/hotel/russia/severodvinsk/...
2,https://ostrovok.ru/hotel/russia/arkhangelsk/m...
3,https://ostrovok.ru/hotel/russia/arkhangelsk/m...
4,https://ostrovok.ru/hotel/russia/arkhangelsk/m...
...,...
2865,https://ostrovok.ru/hotel/russia/vologda/mid10...
2866,https://ostrovok.ru/hotel/russia/vologda/mid10...
2867,https://ostrovok.ru/hotel/russia/vologda/mid90...
2868,https://ostrovok.ru/hotel/russia/cherepovets/m...


In [142]:
options = Options()
options.add_experimental_option('prefs', {
    'profile.managed_default_content_settings.images': 2,
    'profile.managed_default_content_settings.mixed_script': 2,
    'profile.managed_default_content_settings.media_stream': 2,
    'profile.managed_default_content_settings.stylesheets': 2,
        })

def Sertificate(url):
    try:
        
        driver.get(url+'?q=413&dates=10.06.2024-16.06.2024&guests=2&sid=091faba8-2d09-47b2-ad0e-ac86f0e17e8c&lang=ru')

        # Wait for the hover element to be clickable with an increased timeout
        try:
            hover_element = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH,"//*[@id=':0']/div/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[1]/div[2]/div[1]/div[3]/div")))
        except TimeoutException:
            hover_element = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH,"//*[@id=':0']/div/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[1]/div[2]/div[1]/div[2]/div")))
        except:
            return None
        
        # Наводим на него
        ActionChains(driver).move_to_element(hover_element).perform()

        # выгружаем в html. Потом кидаешь это в суп и разбираешь
        html = driver.page_source
        soup = bs(html, 'html.parser')

        sert = soup.find("div", {'class': 'zentexttip-text'})
        if sert:
            return sert.text
        else:
            return None
    except Exception as e:
        return None

In [120]:
url ='https://ostrovok.ru/hotel/russia/arkhangelsk/mid7759870/artelekom_hotel/?q=413&dates=10.06.2024-16.06.2024&guests=2&sid=091faba8-2d09-47b2-ad0e-ac86f0e17e8c&lang=ru'
Sertificate(url)


'The certificate 78/АА-004/029-1/2022. Based on the data from the Federal List of Tourist Sites.'

In [145]:
df2 =pd.DataFrame()

# Define column names
headers = ['URL', 'Sertificate']

# Create an empty DataFrame with headers
df2 = pd.DataFrame(columns=headers)

df2

df3 =pd.DataFrame()

In [146]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

counter = 0

#цикл для заполнения пустого датафрейма df2 значениями о наличии или отсутствии на странице сертификата
for i, url in enumerate(df['URL']):
    
    df2.loc[i, 'URL'] = df.loc[i, "URL"]
    df2.loc[i, 'Sertificate'] = Sertificate(url)
    
    counter += 1
    if counter % 100 == 0:
        df3 = pd.concat([df3, df2]).drop_duplicates(keep=False)
    else:
        pass

# Close the WebDriver instance after use
driver.quit()

df2

,URL,Sertificate
0,https://ostrovok.ru/hotel/russia/arkhangelsk/m...,None
1,https://ostrovok.ru/hotel/russia/severodvinsk/...,Сертификат АА-037/755-20223. По данным из Феде...
2,https://ostrovok.ru/hotel/russia/arkhangelsk/m...,Сертификат 66/03/447-2023. По данным из Федера...
3,https://ostrovok.ru/hotel/russia/arkhangelsk/m...,Сертификат 78/АА-004/029-1/2022. По данным из ...
4,https://ostrovok.ru/hotel/russia/arkhangelsk/m...,Сертификат №78/АА-058-2022/939-2023. По данным...
...,...,...
2865,https://ostrovok.ru/hotel/russia/vologda/mid10...,None
2866,https://ostrovok.ru/hotel/russia/vologda/mid10...,None
2867,https://ostrovok.ru/hotel/russia/vologda/mid90...,None
2868,https://ostrovok.ru/hotel/russia/cherepovets/m...,None


In [147]:
df2.to_excel('Sertificates.xlsx', index=False)


In [148]:
df3

,URL,Sertificate
100,https://ostrovok.ru/hotel/russia/arkhangelsk/m...,None
101,https://ostrovok.ru/hotel/russia/solovetsky/mi...,None
102,https://ostrovok.ru/hotel/russia/arkhangelsk/m...,None
103,https://ostrovok.ru/hotel/russia/arkhangelsk/m...,None
104,https://ostrovok.ru/hotel/russia/solovetsky/mi...,None
...,...,...
2795,https://ostrovok.ru/hotel/russia/cherepovets/m...,None
2796,https://ostrovok.ru/hotel/russia/danilov/mid86...,None
2797,https://ostrovok.ru/hotel/russia/gavrilovskaya...,None
2798,https://ostrovok.ru/hotel/russia/vologda/mid84...,None
